In [2]:
# Imports nécessaires pour tout le script
import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine, text
from snowflake.sqlalchemy import URL
from snowflake.connector.errors import OperationalError

# ======================================================================
#                            CONFIGURATION
# ======================================================================

# A. Infos sur la source (votre base de données locale SQLite)
DB_PATH = '/Users/mory_jr/Library/DBeaverData/workspace6/.metadata/sample-database-sqlite-1/Chinook.db'

# Nom que la table aura dans Snowflake (notre destination)
SNOWFLAKE_TARGET_TABLE = 'ORDERS'

# B. Infos de connexion à Snowflake (À NE PAS PARTAGER !)
SNOWFLAKE_USER = 'MMEITE'
SNOWFLAKE_PASSWORD = 'Junior-23112002@'
SNOWFLAKE_ACCOUNT = 'GAZDZMH-HZ62376' 
SNOWFLAKE_WAREHOUSE = 'COMPUTE_WH'     # L'entrepôt virtuel qui va faire le travail
SNOWFLAKE_DATABASE = 'ETL_PROJECT_DB'  # Le nom de la Base de Données
SNOWFLAKE_SCHEMA = 'SALES_SCHEMA'      # Le nom du Schéma (sous-dossier)

print("Configuration chargée et prête à être utilisée.")

Configuration chargée et prête à être utilisée.


In [3]:
# --- Préparation de la Connexion SQLAlchemy ---

# On construit l'adresse complète pour se connecter à Snowflake
snowflake_url = URL(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE, 
    schema=SNOWFLAKE_SCHEMA
)

try:
    # On crée le moteur qui gère la communication
    engine = create_engine(snowflake_url)
    
    # On ouvre une connexion SÛRE qui se ferme automatiquement grâce au 'with'
    with engine.connect() as conn_snowflake:
        print(f"✅ Connexion à Snowflake établie sur {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.")

        # --- Étape 1 : Création de la Base de Données et du Schéma ---
        print("\n--- Préparation de la DB et du Schéma dans Snowflake ---")

        # Crée la DB et le Schéma s'ils n'existent pas
        conn_snowflake.execute(text(f"CREATE DATABASE IF NOT EXISTS {SNOWFLAKE_DATABASE}"))
        conn_snowflake.execute(text(f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"))
        
        # S'assure d'utiliser le bon entrepôt
        conn_snowflake.execute(text(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}"))
        conn_snowflake.execute(text("COMMIT")) # Enregistre les changements DDL
        
        print(f"   -> Contexte DDL prêt.")

except OperationalError as e:
    print(f"🛑 Erreur critique : Problème d'identifiants ou de réseau. {e}")
except Exception as e:
    print(f"🛑 Erreur lors de la préparation : {e}")

# L'engine et la connexion sont maintenant fermés, ce qui est normal ici.

✅ Connexion à Snowflake établie sur ETL_PROJECT_DB.SALES_SCHEMA.

--- Préparation de la DB et du Schéma dans Snowflake ---
   -> Contexte DDL prêt.


In [4]:
# --- Étape 2 : Extraction et Transformation (E & T) ---

order_df = None # Initialisation du DataFrame

if not os.path.exists(DB_PATH):
    print(f"\n🛑 Erreur: Fichier de base de données SQLite non trouvé à l'adresse: {DB_PATH}")
    order_df = pd.DataFrame() 
else:
    try:
        print("\n--- Extraction de SQLite vers DataFrame ---")
        conn_sqlite = sqlite3.connect(DB_PATH)
        
        # Requête pour extraire toutes les données des factures
        query = "SELECT * FROM Invoice;"
        order_df = pd.read_sql_query(query, conn_sqlite)
        
        print(f"   -> DataFrame 'Invoice' chargé avec {len(order_df)} lignes.")

        # Transformation : On renomme les colonnes pour les faire correspondre à notre table finale 'ORDERS'
        order_df = order_df.rename(
            columns={
                'InvoiceId': 'ORDER_ID', 
                'CustomerId': 'CUSTOMER_ID', 
                'InvoiceDate': 'ORDER_DATE',
                'Total': 'Total' # S'assurer que 'Total' est bien là si elle existe
            }
        )
        print("✅ Transformation: Colonnes clés renommées.")

    except sqlite3.Error as e:
        print(f"🛑 Erreur SQLite lors de l'extraction : {e}")
        order_df = pd.DataFrame()
    finally:
        if 'conn_sqlite' in locals() and conn_sqlite:
            conn_sqlite.close()

if not order_df.empty:
    display(order_df.head())
else:
    print("Aucune donnée à charger.")


--- Extraction de SQLite vers DataFrame ---
   -> DataFrame 'Invoice' chargé avec 412 lignes.
✅ Transformation: Colonnes clés renommées.


,ORDER_ID,CUSTOMER_ID,ORDER_DATE,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [5]:
# --- Étape 3 : Chargement (L) dans Snowflake ---

if order_df is not None and not order_df.empty: 
    
    # Recréer l'adresse de connexion car l'ancienne a été fermée
    snowflake_url = URL(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE, 
        schema=SNOWFLAKE_SCHEMA
    )

    try:
        # On recrée le moteur, puis on ouvre une nouvelle connexion SÛRE avec 'with'
        engine = create_engine(snowflake_url)
        with engine.connect() as conn_snowflake:
            
            print("\n--- Début du Chargement du DataFrame vers Snowflake ---")
            
            # On crée le nom complet de la table: "DB"."SCHEMA".TABLE_NAME (IMPORTANT pour Snowflake)
            qualified_table_name = f'"{SNOWFLAKE_DATABASE}"."{SNOWFLAKE_SCHEMA}".{SNOWFLAKE_TARGET_TABLE}'
            
            # Chargement des données !
            order_df.to_sql(
                qualified_table_name, # On utilise le nom COMPLET pour éviter les erreurs de contexte
                con=conn_snowflake, 
                if_exists='replace', # Crée la table, ou la remplace si elle existe
                index=False,         # N'insère pas l'index du DataFrame dans la table
                chunksize=10000,
                method='multi',
            )
            
            conn_snowflake.execute(text("COMMIT")) # Confirme l'insertion des données
            
            print(f"🎉 Chargement de la table '{SNOWFLAKE_TARGET_TABLE}' terminé avec succès !")

    except Exception as e:
        print(f"\n🛑 Erreur critique lors du chargement des données dans Snowflake : {e}")
        
else:
    print("🛑 Chargement annulé car le DataFrame des commandes est vide.")

print("Fermeture de la connexion Snowflake effectuée (grâce à la clause 'with').")


--- Début du Chargement du DataFrame vers Snowflake ---
🎉 Chargement de la table 'ORDERS' terminé avec succès !
Fermeture de la connexion Snowflake effectuée (grâce à la clause 'with').
